First retrieve all the images from MAST (NOTE: this will take a while to run the first time, but successive runs will detect the files already exist)

In [ ]:
from astroquery.mast import Observations 

files = ['jw02731-o001_t017_nircam_clear-f090w_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f187n_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f200w_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f335m_i2d.fits',
         'jw02731-o001_t017_nircam_clear-f444w_i2d.fits',
         'jw02731-o001_t017_nircam_f444w-f470n_i2d.fits'
        ]


for f in files:
    result = Observations.download_file(f'mast:JWST/product/{f}', base_url="https://mast.stsci.edu/api/v0.1/Download/file")

In [ ]:
import warnings
warnings.simplefilter('ignore')

from jdaviz import Imviz

imviz = Imviz(verbosity='error')
for f in files:
    imviz.load_data(f, data_label=f[31:36])


# Link images by WCS (without affine approximation)
imviz.plugins['Links Control'].link_type = 'WCS'
imviz.plugins['Links Control'].wcs_use_affine = False

imviz.show()

Now we'll use the [plot options plugin API](https://jdaviz.readthedocs.io/en/latest/api/jdaviz.configs.default.plugins.plot_options.plot_options.PlotOptions.html#jdaviz.configs.default.plugins.plot_options.plot_options.PlotOptions) to set the color settings for the various image layers.

In [ ]:
p = imviz.plugins['Plot Options']
p.open_in_tray()

First we'll set the color mode so that each layer is assigned its own color (instead of colormaps).

In [ ]:
p.image_color_mode = 'Monochromatic'

We can now color each layer based on the [NIRCam filters](https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-instrumentation/nircam-filters).  

Here the plot options plugin UI was used to manually find reasonable values for all the layer settings, and then copied into the dictionary so that it could be reproduced programmatically in the notebook.  For example, we can select a layer in the dropdown, use the opacity slider, and then either copy the value from the slider or access it with `p.image_opacity`, and populate the dictionary below.

In [ ]:
img_settings = {'f090w': {'image_color': '#cc7ace',
                          'stretch_vmax': 4,
                          'image_opacity': 0.32,
                          'image_contrast': 0.69,
                          'image_bias': 0.39},
                'f187n': {'image_color': '#66cfef',
                          'stretch_vmin': 0,
                          'stretch_vmax': 16,
                          'image_opacity': 0.4,
                          'image_contrast': 0.94,
                          'image_bias': 0.74},
                'f200w': {'image_color': '#61d3e1',
                          'stretch_vmax': 6,
                          'image_opacity': 0.31,
                          'image_contrast': 2.15,
                          'image_bias': 0.66},
                'f335m': {'image_color': '#afff6b',
                          'stretch_vmin': 4,
                          'stretch_vmax': 27,
                          'image_opacity': 0.23,
                          'image_contrast': 3.13,
                          'image_bias': 0.67},
                'f444w': {'image_color': '#ff767c',
                          'stretch_vmax': 10,
                          'image_opacity': 0.18,
                          'image_contrast': 1.77,
                          'image_bias': 0.56},
                'f470n': {'image_color': '#f7787d',
                          'stretch_vmin': 1,
                          'stretch_vmax': 11,
                          'image_opacity': 0.4,
                          'image_contrast': 2.53,
                          'image_bias': 0.27}}

We can now iterate over this dictionary of settings and use the plot options API to set each of the options.

In [ ]:
for layer, settings in img_settings.items():
    p.layer = f'{layer}[DATA]'
    for k,v in settings.items():
        setattr(p, k, v)

And lastly, we can save to an image using the export plot plugin

In [ ]:
imviz.plugins['Export Plot'].save_figure('carina_rgb.png')